# Notebook used for GAN training and image generation as part of:

*Enhancing Cancer Detection with StyleGAN-2 ADA*

In [ ]:
!nvidia-smi

Wed Nov 29 20:54:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Mounting Google Drive

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print("Drive Mounted")
except:
    print("Drive not Mounted")

Mounted at /content/drive
Drive Mounted


Clone Repo and Install Dependencies

In [2]:
!pip install torch
!git clone https://github.com/NVlabs/stylegan3
!pip install ninja
!pip install pillow
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
from tqdm.notebook import tqdm

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 212 (delta 0), reused 1 (delta 0), pack-reused 207
Receiving objects: 100% (212/212), 4.17 MiB | 9.23 MiB/s, done.
Resolving deltas: 100% (98/98), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.6 MB/s eta 0:00:00


In [ ]:
!ls /content/drive/MyDrive/data/gan/images

Data


Resizing images to all the same size

In [ ]:
'''Loops through the files in an input folder (input_folder), resizes them to a
specified new size (new_size), an adds them to an output folder (output_folder).'''
def resize_images_in_folder(input_folder, output_folder, new_size):
    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)

        # Check if the file is an image
        if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            # Open the image file
            image = Image.open(input_path)

            #Convert to RGB
            image = image.convert('RGB')

            # Resize the image
            resized_image = image.resize(new_size)

            # Generate the output file path
            output_path = os.path.join(output_folder, filename)

            # Save the resized image to the output folder
            resized_image.save(output_path)

            print(f"Resized {filename} and saved to {output_path}")


In [ ]:
#Given the path will clear the data in a given directory if it has been corrupted
def clear_dataset_directory(directory):
  !rm -R directory

Resizing all of our various images.

In [ ]:
#input_folder_path = '/content/drive/My Drive/data/gan/images/Data/train/normal'
#output_folder_path = '/content/drive/My Drive/data/gan/resized_images/normal'
#input_folder_path = '/content/drive/My Drive/data/gan/images/Data/train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa'
#output_folder_path = '/content/drive/My Drive/data/gan/resized_images/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa'
#input_folder_path = '/content/drive/MyDrive/data/gan/images/Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa'
#input_folder_path = '/content/drive/MyDrive/data/gan/images/Data/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib'
#output_folder_path = '/content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib'

new_image_size = (256, 256)  # Replace with your desired dimensions

#resize_images_in_folder(input_folder_path, output_folder_path, new_image_size)

Resized 000021 (8).png and saved to /content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000021 (8).png
Resized 000084 (2).png and saved to /content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000084 (2).png
Resized 000046 (5).png and saved to /content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000046 (5).png
Resized 000040 (10).png and saved to /content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000040 (10).png
Resized 000022 (2).png and saved to /content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000022 (2).png
Resized 000044 (9).png and saved to /content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000044 (9).png
Resized 000051 (4).png and saved to /content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000051 (4).png
Resized 000

We can use StyleGAN's Dataset Creation Tool on the kaggle dataset that is in our drive.

In [ ]:
%cd /content/stylegan3/
!ls

/content/stylegan3
avg_spectra.py	 dnnlib      environment.yml  gui_utils    metrics	training       viz
calc_metrics.py  Dockerfile  gen_images.py    legacy.py    README.md	train.py
dataset_tool.py  docs	     gen_video.py     LICENSE.txt  torch_utils	visualizer.py


The below code checks to make sure our data is all the same size and type. I.E. in the proper format for training our GAN.

In [ ]:
#IMAGE_PATH = '/content/drive/MyDrive/data/gan/resized_images/normal'
#IMAGE_PATH = '/content/drive/My Drive/data/gan/resized_images/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa'

files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz

  0%|          | 0/155 [00:00<?, ?it/s]

Using the StyleGAN Dataset Tool

In [ ]:
#!python dataset_tool.py --source=/content/drive/MyDrive/data/gan/resized_images/normal --dest=/content/drive/MyDrive/data/gan/dataset/tumor/cleaned/normal --resolution='256x256'
#!python dataset_tool.py --source=/content/drive/MyDrive/data/gan/resized_images/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa --dest=/content/drive/MyDrive/data/gan/dataset/tumor/cleaned/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa --resolution='256x256'
#!python dataset_tool.py --source=/content/drive/MyDrive/data/gan/resized_images/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa --dest=/content/drive/MyDrive/data/gan/dataset/tumor/cleaned/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa --resolution='256x256'
#!python dataset_tool.py --source=/content/drive/MyDrive/data/gan/resized_images/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib --dest=/content/drive/MyDrive/data/gan/dataset/tumor/cleaned/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib --resolution='256x256'

100% 195/195 [00:04<00:00, 48.40it/s]


Training the GAN on our data

In [ ]:
# Training
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset/tumor/cleaned/adenocarcinoma"
SNAP = 10
KIMG = 80

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan3/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}  --kimg {KIMG} --cfg stylegan2 --gpus 1 --batch 8 --gamma 50"
!{cmd}

/usr/bin/python3: can't open file '/content/stylegan3/train.py': [Errno 2] No such file or directory


Resuming training from a pickle file

In [ ]:
# Resume Settings
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments/00001-stylegan2-squamous.cell.carcinoma_left-gpus1-batch8-gamma50"
NETWORK = "network-snapshot-000040.pkl"
RESUME = os.path.join(EXPERIMENTS, NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset/tumor/cleaned/squamous.cell.carcinoma/00000"
SNAP = 10
KIMG = 40

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan3/train.py  --snap {SNAP} --kimg {KIMG} --cfg stylegan2 --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA} --gpus 1 --batch 8 --gamma 50"
!{cmd}


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan2.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "fused_modconv_default": "inference_only"
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 50.0,
    "style_mixing_prob": 0.9,
    "pl_weight": 2,
    "pl_no_weight_gra

Image Generation from most recent network model saved to pickle file.

In [4]:
pickle_file = "network-snapshot-000080.pkl"
model_path = f'/content/drive/MyDrive/data/gan/experiments/00000-stylegan2-normal-gpus1-batch8-gamma50/{pickle_file}'
!python /content/stylegan3/gen_images.py --outdir=/content/drive/MyDrive/data/synthetic --trunc=1 --seeds=49-200 \
    --network=$model_path

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00000-stylegan2-normal-gpus1-batch8-gamma50/network-snapshot-000080.pkl"...
Generating image for seed 49 (0/152) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 50 (1/152) ...
Generating image for seed 51 (2/152) ...
Generating image for seed 52 (3/152) ...
Generating image for seed 53 (4/152) ...
Generating image for seed 54 (5/152) ...
Generating image for seed 55 (6/152) ...
Generating image for seed 56 (7/152) ...
Generating image for seed 57 (8/152) ...
Generating image for seed 58 (9/152) ...
Generating image for seed 59 (10/152) ...
Generating image for seed 60 (11/152) ...
Generating image for seed 61 (12/152) ...
Generating image for seed 62 (13/152) ...
Generating image for seed 63 (14/152) ...
Generating image for seed 64 (15/152) ...
Generating image for seed 65 (16/152) ...
Generating image for seed 66 (17/152) ...
Gene

In [3]:
model_path = f'/content/drive/MyDrive/data/gan/experiments/00001-stylegan2-squamous.cell.carcinoma_left-gpus1-batch8-gamma50/00001-stylegan2-00000-gpus1-batch8-gamma50/network-snapshot-000040.pkl'
!python /content/stylegan3/gen_images.py --outdir=/content/drive/MyDrive/data/synthetic/squamous --trunc=1 --seeds=0-200 \
    --network=$model_path

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00001-stylegan2-squamous.cell.carcinoma_left-gpus1-batch8-gamma50/00001-stylegan2-00000-gpus1-batch8-gamma50/network-snapshot-000040.pkl"...
Generating image for seed 0 (0/201) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 1 (1/201) ...
Generating image for seed 2 (2/201) ...
Generating image for seed 3 (3/201) ...
Generating image for seed 4 (4/201) ...
Generating image for seed 5 (5/201) ...
Generating image for seed 6 (6/201) ...
Generating image for seed 7 (7/201) ...
Generating image for seed 8 (8/201) ...
Generating image for seed 9 (9/201) ...
Generating image for seed 10 (10/201) ...
Generating image for seed 11 (11/201) ...
Generating image for seed 12 (12/201) ...
Generating image for seed 13 (13/201) ...
Generating image for seed 14 (14/201) ...
Generating image for seed 15 (15/201) ...
Generating image for seed 16 (16/